Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Notes from dataset

In [2]:
import pandas as pd

notes_path = '/content/drive/MyDrive/NOTEEVENTS.csv'

df = pd.read_csv(notes_path)

<ipython-input-2-e0103223d09c>:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(notes_path)


In [3]:
df.sample(5)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1735191,1708498,2218,153622.0,2136-02-12,2136-02-12 13:44:00,2136-02-12 13:48:00,Nursing/other,Report,17593.0,NaN,Neonatology Attending\n\nDOL 111 CGA 41 2/7 we...
804054,812093,19240,NaN,2145-11-26,2145-11-26 10:37:00,NaN,Radiology,L UNILAT LOWER EXT VEINS LEFT,NaN,NaN,[**2145-11-26**] 10:37 AM\n UNILAT LOWER EXT V...
1432512,1465074,15633,147072.0,2157-12-14,2157-12-14 15:13:00,2157-12-14 15:24:00,Nursing/other,Report,14387.0,NaN,FULL CODE Universal Precautions NKDA\n\n...
1782008,1802073,8797,127928.0,2175-08-30,2175-08-30 16:20:00,2175-08-30 16:24:00,Nursing/other,Report,18714.0,NaN,Neonatology Fellow Note\nExam:\nGeneral: pink ...
808326,799847,18057,138573.0,2143-08-30,2143-08-30 16:30:00,NaN,Radiology,INITAL 2ND ORDER ABD/PEL/LOWER EXT A-GRAM,NaN,NaN,[**2143-8-30**] 4:30 PM\n [**Hospital1 **]-LAT...


In [4]:
# Get all unique patient ids

print(df.shape)
patients = df['SUBJECT_ID'].unique()
print(df['CATEGORY'].unique())

(2083180, 11)
['Discharge summary' 'Echo' 'ECG' 'Nursing' 'Physician ' 'Rehab Services'
 'Case Management ' 'Respiratory ' 'Nutrition' 'General' 'Social Work'
 'Pharmacy' 'Consult' 'Radiology' 'Nursing/other']


In [23]:
# pick a random patient

import numpy as np

sample = np.random.choice(patients)

patient = df[df['SUBJECT_ID'] == sample]

print(patient.shape)


(11, 11)


In [24]:
# Extract Dishcharge summary

discharge = patient[patient['CATEGORY'] == 'Discharge summary']
try:
  report = discharge['TEXT']

  text = report.iloc[0]

  print(discharge.shape)

except:
  print('No Discharge Summary')
  types = patient['CATEGORY'].unique()
  print(types)
  for t in types:
    print(t)
    print(patient[patient['CATEGORY'] == t].shape)

(2, 11)


In [65]:
def check_text(text):
  keyWords = ['laceration', 'lacerations', 'abrasion', 'abrasions','burns', 'burn', 'necrotic tissue', 'granular tissue', 'granulation tissue', 'cut', 'cuts', 'bandage', 'degloving', 'wound','wounds', 'fractures']

  words = []
  for word in keyWords:
    if word in text:
      words.append(word)
  if len(words):
    return True, words
  else:
    return False, words

In [66]:
#Filter patients by Dishcarge summary that contains 3 or more keywords



count = 0
no_discharge = 0
good_patients = []
# sample = np.random.choice(patients, 1000)
for patient in patients:
  text = ''
  df_patient = df[df['SUBJECT_ID'] == patient]
  discharge = df_patient[df_patient['CATEGORY'] == 'Discharge summary']
  try:
    report = discharge['TEXT']

    for i in range(report.shape[0]):
      text += report.iloc[i]

    check = check_text(text)
    if(check[0]):
      if(len(check[1]) > 2):
        count += 1
        # print(f'{patient}--{check[1]}')
        good_patients.append(patient)

  except:
    no_discharge += 1

print(count)
print(no_discharge)

5082
0


In [77]:
def get_discharge(patient_id):
  extra = df[df['SUBJECT_ID'] ==patient_id ]
  text =''

  d = extra[extra['CATEGORY']=='Discharge summary']

  for i in range(d.shape[0]):
    words = check_text(d['TEXT'].iloc[i])
    if len(words[1]) > 2:
      print(words[1])
      text += d['TEXT'].iloc[i]


  return text

In [78]:
# clean text
def clean_text(text):
  import re

  text_alpha = re.sub(r'\[(.*?)\]',' ', text)
  text_upper = text_alpha.upper()

  return text_upper


ran = np.random.choice(good_patients)

clean_discharge = clean_text(get_discharge(ran))


['cut', 'wound', 'wounds']


In [75]:
# Get Gemini API ke

from google.colab import userdata
import google.generativeai as genai

api_key = userdata.get('gemini')
client = genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")


In [83]:
def set_prompt(clean_discharge):

  prompt = f"""### Instruction ###
  Please summarize the following discharge summary for a critical care patient, focusing on their injuries and the specific treatment plan implemented during their stay and for discharge.

  Specifically, extract the following information:

  **Injuries:**
  - List all specific injuries mentioned, including location and severity if available.
  - Provide the specifics of what caused injury.

  **Treatment Plan:**
  - **Medications:** List all medications prescribed at discharge, including name, dosage, route, and frequency.
  - **Procedures:** List all specific medical or nursing procedures performed during the stay and any instructions for ongoing procedures (e.g., wound care, dressing changes).
  - **Physical Therapy/Rehabilitation:** If mentioned, summarize any physical therapy or rehabilitation plans, including exercises and frequency.
  - **Follow-up:** Note any scheduled follow-up appointments or instructions for monitoring.

  ### Discharge Summary ###
  {clean_discharge}

  ### Summary ###
  ### Injuries ###
  ### Treatment Plan ###
  ### Follow-up ###
  """
  return prompt

In [85]:

import time

batch = 0
with open('rag_doc.txt', 'w') as out:
  for patient in good_patients:
    batch += 1
    prompt = set_prompt(clean_text(get_discharge(patient)))
    response = model.generate_content(prompt)
    out.write('New Patient:\n')
    out.write(response.text)
    if batch % 15 == 0:
      time.sleep(60)

['cut', 'wound', 'wounds']
['laceration', 'cut', 'fractures']
['laceration', 'cut', 'wound', 'fractures']
['laceration', 'cut', 'fractures']
['cut', 'wound', 'wounds']
['laceration', 'abrasion', 'abrasions', 'cut', 'fractures']
['laceration', 'wound', 'fractures']
['burn', 'cut', 'wound']
['cut', 'wound', 'wounds']
['cut', 'wound', 'wounds']
['cut', 'wound', 'wounds']
['cut', 'bandage', 'wound']
['cut', 'wound', 'wounds']
['cut', 'wound', 'wounds']
['laceration', 'granulation tissue', 'wound', 'wounds', 'fractures']
['granulation tissue', 'wound', 'wounds']
['burn', 'cut', 'wound']
['abrasion', 'abrasions', 'cut']
['burn', 'cut', 'wound', 'wounds']
['cut', 'wound', 'wounds']
['abrasion', 'abrasions', 'burn', 'cut']
['burn', 'cut', 'wound']
['burn', 'cut', 'wound']
['cut', 'wound', 'wounds']
['laceration', 'cut', 'wound']
['cut', 'wound', 'wounds', 'fractures']
['granulation tissue', 'cut', 'wound']
['laceration', 'cut', 'fractures']
['cut', 'wound', 'fractures']
['cut', 'wound', 'fract

['cut', 'wound', 'wounds']
['cut', 'wound', 'wounds']


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.